In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from collections import Counter

In [2]:
source_info = pd.read_csv('../data/SourceInfo.csv')
caa_c0=pd.read_csv("../data/caa_nrc_cluster_0.csv")
caa_c1=pd.read_csv("../data/caa_nrc_cluster_1.csv")
caa_c2=pd.read_csv("../data/caa_nrc_cluster_2.csv")
covid_c0=pd.read_csv("../data/covid_cluster_0.csv")
covid_c1=pd.read_csv("../data/covid_cluster_1.csv")
covid_c2=pd.read_csv("../data/covid_cluster_2.csv")
original_parsed_data = pd.read_csv("../data/parsed_cleaned.csv")
strong_words_data = pd.read_csv("../data/StrongWordsAnalysis.csv")

In [24]:
caa_c0.head()

,Unnamed: 0,Title,Text,sentiment,neutrality
0,0,"Visit Shaheen Bagh, listen to concerns on CAA,...",woman writes message postcard addressed pm pm ...,-0.2023,0.026270
1,1,CAA-NRC-NPR chronology will make people spend ...,download time news app latest news subscribe s...,-0.8294,0.023000
2,2,"JNU students protest against CAA, NRC at Janta...",university student union president black cloth...,-0.8074,0.019782
3,3,caa/nrc: CAA-NRC: Ludhiana to have Shaheen Bag...,white beard download time news app latest city...,-0.8834,0.031924
4,4,"Indefinite protest over CAA, NRC starts at Dan...",download time news app latest city news subscr...,-0.4019,0.003872


In [3]:
Counter(original_parsed_data['Source'])

Counter({'The Times of India': 68,
         'Indian Express': 128,
         'The Wire': 65,
         'The Hindu': 64,
         'News18': 70,
         'FirstPost': 65,
         'DNA': 63,
         'NDTV': 61,
         'DDNews': 55,
         'Rajya Sabha TV': 61,
         'The Quint': 69,
         'Times Now': 58,
         'The Print': 60,
         'Hindustan Times': 61,
         'India Today': 61,
         'Swarajya': 62,
         'Organizer': 58,
         'National Herald': 57,
         'Deccan Chronicle': 60,
         'Republic TV': 59,
         'The Statesman': 62,
         'Scroll': 61,
         'Outlook': 45,
         'Independent': 50,
         'Janta Ka Reporter': 60,
         'News Laundry': 49,
         'OpIndia': 66,
         'Reuters': 48,
         'The Telegraph': 60,
         'Zee News': 60})

In [4]:
articles_copy = caa_c0
articles_copy = articles_copy.append([caa_c1,caa_c2,covid_c0,covid_c1,covid_c2])

In [5]:
articles_copy

,Unnamed: 0,Title,Text,sentiment,neutrality
0,0,"Visit Shaheen Bagh, listen to concerns on CAA,...",woman writes message postcard addressed pm pm ...,-0.2023,0.026270
1,1,CAA-NRC-NPR chronology will make people spend ...,download time news app latest news subscribe s...,-0.8294,0.023000
2,2,"JNU students protest against CAA, NRC at Janta...",university student union president black cloth...,-0.8074,0.019782
3,3,caa/nrc: CAA-NRC: Ludhiana to have Shaheen Bag...,white beard download time news app latest city...,-0.8834,0.031924
4,4,"Indefinite protest over CAA, NRC starts at Dan...",download time news app latest city news subscr...,-0.4019,0.003872
...,...,...,...,...,...
144,144,"Rahul Gandhi questions COVID-19 testing speed,...",new congress leader reiterated stand enough te...,-0.8910,0.375725
145,145,"Amid COVID-19 spread, Air India cancels all bo...",pm pm air cancelled booking booking closed til...,0.9744,0.105955
146,146,"Amid coronavirus COVID-19 crisis, PM Narendra ...",amid day nationwide due coronavirus covid pand...,0.5719,0.015132
147,147,COVID-19 spread: Goal to ensure minimum loss o...,prime minister outlined common goal country en...,0.9900,0.111126


In [6]:
articles = []
for i in original_parsed_data.index:
    source=original_parsed_data['Source'].iloc[i]
    title=original_parsed_data['Title'].iloc[i]
    topic=original_parsed_data['Topic Name'].iloc[i]
    text=original_parsed_data['Cleaned_text'].iloc[i]
    source_reliability=source_info.loc[source_info['Source']==source,'SourceScore'].iloc[0]
    strong_words_score = strong_words_data.loc[strong_words_data['Title']==title,'strongP'].iloc[0]
    neutrality_score=articles_copy['neutrality'].iloc[i]
    articles.append([source,title,topic,text,source_reliability,strong_words_score,neutrality_score])

In [7]:
articles_data = pd.DataFrame(articles,columns=["Source","Title","Topic Name","Cleaned Text","Source Score","Strong Words Score","Neutrality Score"])
articles_data.head()

,Source,Title,Topic Name,Cleaned Text,Source Score,Strong Words Score,Neutrality Score
0,The Times of India,"PM Modi, Amit Shah must engage in dialogue ove...",caa-nrc,download time news app latest news subscribe s...,1.0,1.0,0.026270
1,The Times of India,"Trump to discuss CAA, NRC issues with Modi dur...",caa-nrc,united state president trump prime minister fi...,1.0,3.0,0.023000
2,The Times of India,"Sonia Gandhi: PM Modi, Amit Shah misled people...",caa-nrc,download time news app latest news subscribe s...,1.0,1.0,0.019782
3,The Times of India,"I told PM Modi to withdraw CAA, NRC and NPR: M...",caa-nrc,story download time news app latest news subsc...,1.0,3.0,0.031924
4,The Times of India,PM Modi's visit to Bangladesh: Simmering tensi...,caa-nrc,prime minister file photo download time news a...,1.0,2.0,0.003872


In [8]:
articles_data['Reliability Score']=""
for i in articles_data.index:
    score = (0.35)*articles_data['Source Score'][i]+(0.35)*articles_data['Neutrality Score'][i]+(0.3)*articles_data['Strong Words Score'][i]
    articles_data['Reliability Score'][i] = round(score,3)

C:\Users\vogir\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [9]:
a=max(articles_data['Reliability Score'])
for i in range(len(articles_data)):
    b=articles_data['Reliability Score'][i]/a
    articles_data['Reliability Score'][i]=b

C:\Users\vogir\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [10]:
articles_data['Reliability Label']=""
for i in articles_data.index:

    if articles_data['Reliability Score'][i]>=0.6:
        articles_data['Reliability Label'][i]='Reliable'
    else:
        articles_data['Reliability Label'][i]='Not Reliable'

C:\Users\vogir\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\vogir\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [11]:
articles_data.head()

,Source,Title,Topic Name,Cleaned Text,Source Score,Strong Words Score,Neutrality Score,Reliability Score,Reliability Label
0,The Times of India,"PM Modi, Amit Shah must engage in dialogue ove...",caa-nrc,download time news app latest news subscribe s...,1.0,1.0,0.026270,0.454483,Not Reliable
1,The Times of India,"Trump to discuss CAA, NRC issues with Modi dur...",caa-nrc,united state president trump prime minister fi...,1.0,3.0,0.023000,0.867586,Reliable
2,The Times of India,"Sonia Gandhi: PM Modi, Amit Shah misled people...",caa-nrc,download time news app latest news subscribe s...,1.0,1.0,0.019782,0.453103,Not Reliable
3,The Times of India,"I told PM Modi to withdraw CAA, NRC and NPR: M...",caa-nrc,story download time news app latest news subsc...,1.0,3.0,0.031924,0.869655,Reliable
4,The Times of India,PM Modi's visit to Bangladesh: Simmering tensi...,caa-nrc,prime minister file photo download time news a...,1.0,2.0,0.003872,0.655862,Reliable


In [12]:
l1 = list(articles_data['Source Score'])
l2 = list(articles_data['Strong Words Score'])
l3 = list(articles_data['Neutrality Score'])
l4 = list(articles_data['Reliability Score'])
print(max(l1),min(l1))
print(max(l2),min(l2))
print(max(l3),min(l3))
print(max(l4),min(l4))

1.0 0.16056338028169018
3.0 1.0
0.5823095752547777 4.659997040383821e-08
1.0 0.26620689655172414


In [13]:
reliable_articles=articles_data[articles_data['Reliability Label']=='Reliable']
unreliable_articles=articles_data[articles_data['Reliability Label']=='Not Reliable']
s1=Counter(reliable_articles['Source'])
s2=Counter(unreliable_articles['Source'])

print(s1)
print()
print(s2)

Counter({'Indian Express': 88, 'The Hindu': 62, 'The Times of India': 61, 'Hindustan Times': 59, 'The Quint': 53, 'The Statesman': 53, 'India Today': 48, 'Reuters': 48, 'DDNews': 43, 'Zee News': 43, 'FirstPost': 41, 'DNA': 41, 'Rajya Sabha TV': 41, 'The Telegraph': 41, 'News18': 40, 'The Print': 40, 'Swarajya': 40, 'Independent': 38, 'NDTV': 37, 'Republic TV': 37, 'Times Now': 35, 'National Herald': 34, 'Deccan Chronicle': 32, 'OpIndia': 30, 'News Laundry': 28, 'Outlook': 27, 'Janta Ka Reporter': 26, 'Scroll': 24, 'The Wire': 23, 'Organizer': 21})

Counter({'The Wire': 42, 'Indian Express': 40, 'Organizer': 37, 'Scroll': 37, 'OpIndia': 36, 'Janta Ka Reporter': 34, 'News18': 30, 'Deccan Chronicle': 28, 'FirstPost': 24, 'NDTV': 24, 'Times Now': 23, 'National Herald': 23, 'DNA': 22, 'Swarajya': 22, 'Republic TV': 22, 'News Laundry': 21, 'Rajya Sabha TV': 20, 'The Print': 20, 'The Telegraph': 19, 'Outlook': 18, 'Zee News': 17, 'The Quint': 16, 'India Today': 13, 'DDNews': 12, 'Independent'

In [14]:
Counter(articles_data['Reliability Label'])
s1=dict(s1)
s2=dict(s2)
s2

{'The Times of India': 7,
 'Indian Express': 40,
 'The Wire': 42,
 'The Hindu': 2,
 'News18': 30,
 'FirstPost': 24,
 'DNA': 22,
 'NDTV': 24,
 'DDNews': 12,
 'Rajya Sabha TV': 20,
 'The Quint': 16,
 'Times Now': 23,
 'The Print': 20,
 'Hindustan Times': 2,
 'India Today': 13,
 'Swarajya': 22,
 'Organizer': 37,
 'National Herald': 23,
 'Deccan Chronicle': 28,
 'Republic TV': 22,
 'The Statesman': 9,
 'Scroll': 37,
 'Outlook': 18,
 'Independent': 12,
 'Janta Ka Reporter': 34,
 'News Laundry': 21,
 'OpIndia': 36,
 'The Telegraph': 19,
 'Zee News': 17}

In [15]:
Counter(articles_data['Reliability Label'])

Counter({'Not Reliable': 632, 'Reliable': 1234})

In [16]:
source_info['Reliability Score']=""
for i in range(len(source_info)):
    if source_info['Source'][i] not in s1:
        source_info['Reliability Score'][i] = 0
    elif source_info['Source'][i] not in s2:
        source_info['Reliability Score'][i] = 1
    else:
        source_info['Reliability Score'][i] = s1[source_info['Source'][i]]/(s2[source_info['Source'][i]]+s1[source_info['Source'][i]])

C:\Users\vogir\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\vogir\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
source_info.sort_values(by='Reliability Score')

,Source,Wikipedia,Year,Twitter,Verified,Popularity,History,Controversy,SourceScore,Reliability Score
26,The Wire,1,2015,1,1,Low,1,1,0.292958,0.353846
14,Organizer,1,1947,1,0,Low,1,0,0.450704,0.362069
19,Scroll,1,2014,1,1,Low,1,0,0.276056,0.393443
8,Janta Ka Reporter,0,2015,1,1,Low,0,0,0.160563,0.433333
13,OpIndia,1,2014,1,1,Medium,1,0,0.380282,0.454545
1,Deccan Chronicle,1,1938,1,1,Low,0,0,0.433803,0.533333
11,News Laundry,0,2012,1,1,Low,0,0,0.169014,0.571429
12,News18,1,2005,1,1,High,1,1,0.529577,0.571429
9,National Herald,1,2017,1,1,Low,1,1,0.287324,0.596491
15,Outlook,1,1995,1,1,Low,1,0,0.329577,0.6


In [22]:
source_info

,Source,Wikipedia,Year,Twitter,Verified,Popularity,History,Controversy,SourceScore,Reliability Score,Reliability Label
0,DDNews,1,2003,1,1,High,1,0,0.515493,0.781818,Reliable
1,Deccan Chronicle,1,1938,1,1,Low,0,0,0.433803,0.533333,Not Reliable
2,DNA,0,2008,1,1,Low,0,0,0.180282,0.650794,Reliable
3,FirstPost,1,2017,1,1,Low,0,0,0.211268,0.630769,Reliable
4,Hindustan Times,1,1924,1,1,High,1,1,0.757746,0.967213,Reliable
5,Independent,1,1986,1,1,Medium,1,0,0.459155,0.76,Reliable
6,India Today,1,2003,1,1,High,1,0,0.515493,0.786885,Reliable
7,Indian Express,1,1932,1,1,Medium,1,0,0.611268,0.6875,Reliable
8,Janta Ka Reporter,0,2015,1,1,Low,0,0,0.160563,0.433333,Not Reliable
9,National Herald,1,2017,1,1,Low,1,1,0.287324,0.596491,Not Reliable


In [18]:
source_info['Reliability Label']=""
for i in range(len(source_info)):

    if source_info['Reliability Score'][i]>0.6:
        source_info['Reliability Label'][i]='Reliable'
    else:
        source_info['Reliability Label'][i]='Not Reliable'

C:\Users\vogir\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\vogir\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [19]:
source_info

,Source,Wikipedia,Year,Twitter,Verified,Popularity,History,Controversy,SourceScore,Reliability Score,Reliability Label
0,DDNews,1,2003,1,1,High,1,0,0.515493,0.781818,Reliable
1,Deccan Chronicle,1,1938,1,1,Low,0,0,0.433803,0.533333,Not Reliable
2,DNA,0,2008,1,1,Low,0,0,0.180282,0.650794,Reliable
3,FirstPost,1,2017,1,1,Low,0,0,0.211268,0.630769,Reliable
4,Hindustan Times,1,1924,1,1,High,1,1,0.757746,0.967213,Reliable
5,Independent,1,1986,1,1,Medium,1,0,0.459155,0.76,Reliable
6,India Today,1,2003,1,1,High,1,0,0.515493,0.786885,Reliable
7,Indian Express,1,1932,1,1,Medium,1,0,0.611268,0.6875,Reliable
8,Janta Ka Reporter,0,2015,1,1,Low,0,0,0.160563,0.433333,Not Reliable
9,National Herald,1,2017,1,1,Low,1,1,0.287324,0.596491,Not Reliable


In [20]:
print("Number of reliable sources")
Counter(source_info['Reliability Label'])

Number of reliable sources


Counter({'Reliable': 20, 'Not Reliable': 10})

In [23]:
articles_data.head()

,Source,Title,Topic Name,Cleaned Text,Source Score,Strong Words Score,Neutrality Score,Reliability Score,Reliability Label
0,The Times of India,"PM Modi, Amit Shah must engage in dialogue ove...",caa-nrc,download time news app latest news subscribe s...,1.0,1.0,0.026270,0.454483,Not Reliable
1,The Times of India,"Trump to discuss CAA, NRC issues with Modi dur...",caa-nrc,united state president trump prime minister fi...,1.0,3.0,0.023000,0.867586,Reliable
2,The Times of India,"Sonia Gandhi: PM Modi, Amit Shah misled people...",caa-nrc,download time news app latest news subscribe s...,1.0,1.0,0.019782,0.453103,Not Reliable
3,The Times of India,"I told PM Modi to withdraw CAA, NRC and NPR: M...",caa-nrc,story download time news app latest news subsc...,1.0,3.0,0.031924,0.869655,Reliable
4,The Times of India,PM Modi's visit to Bangladesh: Simmering tensi...,caa-nrc,prime minister file photo download time news a...,1.0,2.0,0.003872,0.655862,Reliable
